# INTRODUCTION

## Import Libraries and Packages

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
sns.set()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Function Definitions
* ***processDataFrame(file_name)***: takes a file name, loads data in CSV format, and outputs a data frame
* ***dataCleaning(df)***: takes a data frame, encodes categorical variables, imputes missing values, and outputs the updated data frame
* ***countplotConstructor(df)***: takes a data frame and constructs countplots for the categorical variables
* ***histogramConstructor(df)***: takes a data frame, constructs histograms for the quantitative variables, and fits a kernel density estimate to the histogram
* ***scatterplotConstructor(df, a, c)***: takes a data frame, transparency parameter, and a color parameter; it creates six scatter plots, one per predictor variable
* ***printMetrics(y_test, predictions)***: takes the actual and the predicted diamond prices and calculates some relevant performance metrics
* ***residualPlotConstructor(y_test, predictions, a, c1, c2)***: takes the actual and predicted diamond prices, a transparency parameter, and two color parameters; it constructs a residual plot

In [ ]:
def processDataFrame(file_name):
    df = pd.read_csv(file_name)
    print(df.info())
    print(df.describe())
    
    return df
    
def dataCleaning(df):
    cut_map = {'Fair':1, 'Good':2, 'Very Good':3, 'Premium':4, 'Ideal':5}
    color_map = {'D':7, 'E':6, 'F':5, 'G':4, 'H':3, 'I':2, 'J':1}
    clarity_map = {'I1':1, 'SI1':2, 'SI2':3, 'VS1':4, 'VS2':5, 'VVS1':6, 'VVS2':7, 'IF':8}
        
    df.cut = df.cut.replace(cut_map)
    df.color = df.color.replace(color_map)
    df.clarity = df.clarity.replace(clarity_map)
    
    df.x[df.x == 0.0] = df.x.mean()
    df.y[df.y == 0.0] = df.y.mean()
    df.z[df.z == 0.0] = df.z.mean()
    
    df.dropna()
    df.reset_index(drop = True)
    
    return df

def countplotConstructor(df):
    fig, axes = plt.subplots(nrows = 3, figsize = (15, 15))
    sns.countplot(x = 'cut', data = df, ax = axes[0], order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'])
    sns.countplot(x = 'color', data = df, ax = axes[1], order = ['J', 'I', 'H', 'G', 'F', 'E', 'D'])
    sns.countplot(x = 'clarity', data = df, ax = axes[2], order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF'])
    
    plt.tight_layout()
    
def histogramConstructor(df):
    columns = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
    fig, axes = plt.subplots(nrows = len(columns), figsize = (18, 18))
    for i in range(len(columns)):
        sns.distplot(df[columns[i]], kde = True, ax = axes[i])
        axes[i].set(xlabel = columns[i], ylabel = 'frequency', title = columns[i])
    
    plt.tight_layout()    

def boxplotConstructor(df):
    columns = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
    fig, axes = plt.subplots(nrows = len(columns), figsize = (18, 18))
    for i in range(len(columns)):
        sns.boxplot(df[columns[i]], ax = axes[i])
    
    plt.tight_layout()    

def scatterplotConstructor(df, ylabel, a, c):
    fig, axes = plt.subplots(nrows = 2, ncols = 3, figsize = (18, 18))
    axes[0, 0].scatter(df.carat, df.price, alpha = a, color = c)
    axes[0, 1].scatter(df.depth, df.price, alpha = a, color = c)
    axes[0, 2].scatter(df.table, df.price, alpha = a, color = c)
    axes[1, 0].scatter(df.x, df.price, alpha = a, color = c)
    axes[1, 1].scatter(df.y, df.price, alpha = a, color = c)
    axes[1, 2].scatter(df.z, df.price, alpha = a, color = c)

    axes[0, 0].set(xlabel = 'carat', ylabel = ylabel, title = 'price vs carat')
    axes[0, 1].set(xlabel = 'depth', ylabel = ylabel, title = 'price vs depth')
    axes[0, 2].set(xlabel = 'table', ylabel = ylabel, title = 'price vs table')
    axes[1, 0].set(xlabel = 'x-dimension', ylabel = ylabel, title = 'price vs x-dimension')
    axes[1, 1].set(xlabel = 'y-dimension', ylabel = ylabel, title = 'price vs y-dimension')
    axes[1, 2].set(xlabel = 'z-dimension', ylabel = ylabel, title = 'price vs z-dimension')

    plt.tight_layout()

def printMetrics(y_test, predictions):
    mae = metrics.mean_absolute_error(y_test, predictions)
    mse = metrics.mean_squared_error(y_test, predictions)
    mape = np.mean(np.abs(y_test - predictions) / np.abs(y_test))
    
    print('Mean Absolute Error: ', mae)
    print('Mean Squared Error: ', mse)
    print('Root Mean Squared Error: ', np.sqrt(mse))
    print('Mean Absolute Percentage Error: ', round(mape * 100, 2), '%')
    print('Accuracy: ', round(100 * (1 - mape), 2), '%')
    
def residualPlotConstructor(y_test, predictions, a, c1, c2):
    fig, ax = plt.subplots()
    ax.set(xlabel = 'Predicted Price', ylabel = 'Actual Price - Predicted Price', title = 'Residual Plot')
    ax.scatter(predictions, y_test - predictions, alpha = a, color = c1)
    ax.axhline(y = 0, color = c2)

# THE DATA

## Data Exploration
The data frame has a 53940x10 shape. There are no obvious missing values. However, a brief inspection of the statistical summary shows that there are some hidden missing values. The minimum values for the variables ***x***, ***y***, ***z*** are ***0.0***. Since the three variables represent physical dimensions, a measure of ***0.0*** does not make sense. The ***0.0*** is likely a placeholder for missing values. Compared to the overall size of the data set, the number of missing values is negligible; however, I still imputed the missing values by replacing the ***0.0***'s in each column with the average of the corresponding variable.

In [ ]:
file_name = '/kaggle/input/diamonds/diamonds.csv'
df = processDataFrame(file_name)
df[df == 0.0].count(axis = 0)

The majority of the variables — except for ***depth*** — are highly correlated with ***price***. 

In [ ]:
print(df.cut.unique())
print(df.color.unique())
print(df.clarity.unique())
countplotConstructor(df)

In [ ]:
df.corr()

I retained all predictors except for ***Unnamed: 0*** (the index column) and ***depth***.

In [ ]:
df = dataCleaning(df)
y = df.price
X = df.drop(columns = ['Unnamed: 0', 'depth', 'price'])

### The variables ***carat,*** ***price,*** and ***x*** are skewed to the right. The distributions of the variables ***table,*** ***y,*** and ***z*** are jagged. The variable ***depth*** is fairly symmetric, but even that distribution has long tails on each side, suggesting potential extreme values.

In [ ]:
histogramConstructor(df)

### The boxplots make more explicit what the histograms and kde's suggest. Each variable has a considerable number of outliers, either on one or both ends of the distribution. There is nothing about the data to suggest that the outliers are typos or incorrect in some other way. Removing them might lead to a better model fit, but doing so would also remove some of the nuance and variation of the data. Regular multivariate regression and other techniques can be sensitive to extreme values. To avoid this problem, I retained the outliers but used a random forest regressor, which is more robust to outliers.

In [ ]:
boxplotConstructor(df)

In [ ]:
scatterplotConstructor(df, 'price', a = 0.2, c = 'tan')

# THE MODEL

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)
predictions = regressor.predict(X_test)

## Performance Metrics & Residual Plot

In [ ]:
printMetrics(y_test, predictions)
residualPlotConstructor(y_test, predictions, 0.15, 'firebrick', 'maroon')

For low predicted prices, the residuals are closely centered around y = 0. As predicted price increases, the residuals start diverging from y = 0, but still mostly concentrate around the horizontal line.